In [ ]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import time

## Getting all plants in NW USA:

In [ ]:
finaldf = pd.DataFrame()

for i in range(1, 769):
  token='eubVXJDnYoqGNlCnfeRTSLzziXK5QEga2Jjf5O7i75s'
  r = requests.get('https://trefle.io/api/v1/distributions/73/plants?token='+token+'&page='+str(i))
  data = r.json()
  df = json_normalize(data['data'])
  finaldf = pd.concat([finaldf, df])

  time.sleep(0.5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


## Cleaning up links to plant info:

In [ ]:
finaldf = finaldf.reset_index()
finaldf.replace("%C3%97", "x", regex=True, inplace=True)
finaldf.drop_duplicates(subset=["links.plant"], inplace=True)
finaldf["links.self"].replace("species", "plants", regex=True, inplace=True)
finaldf.reset_index(inplace=True)

## Getting detailed information about each plant by doing an API request using its individual link:

In [ ]:
nw_plants = pd.DataFrame()
for i in range(len(finaldf)):
  try:
    link = finaldf.iloc[i]['links.plant']
    token = 'eubVXJDnYoqGNlCnfeRTSLzziXK5QEga2Jjf5O7i75s'
    r = requests.get('https://trefle.io'+link+'?token='+token)
    data = r.json()
    df = json_normalize(data['data'])
    nw_plants = pd.concat([nw_plants, df])
  except KeyError:
    try:
      link = finaldf.iloc[i]['links.self']
      token = 'eubVXJDnYoqGNlCnfeRTSLzziXK5QEga2Jjf5O7i75s'
      r = requests.get('https://trefle.io'+link+'?token='+token)
      data = r.json()
      df = json_normalize(data['data'])
      nw_plants = pd.concat([nw_plants, df])
    except:
      pass
    
  time.sleep(0.5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


## Split up reading the plants into 3 sections due to time constraints and converted each part to csv.

In [ ]:
nw_plantspart1 = pd.read_csv('/content/nw_plantspart1.csv')
nw_plantspart2 = pd.read_csv('/content/nw_plantspart2.csv')
nw_plantspart3 = pd.read_csv('/content/nw_plantspart3.csv')

## Concatenated csvs into one full df, dropped all the columns that were just common-names in different languages.

In [ ]:
nw_plants = pd.concat([nw_plantspart1, nw_plantspart2, nw_plantspart3], ignore_index=True)

In [ ]:
nw_plants = nw_plants[nw_plants.columns.drop(list(nw_plants.filter(regex='common_names')))]

In [ ]:
nw_plants.to_csv('nw_plants.csv')

In [ ]:
nw_plants.head()

,Unnamed: 0,index,id,common_name,slug,scientific_name,main_species_id,image_url,year,bibliography,author,family_common_name,genus_id,observations,vegetable,species,subspecies,varieties,hybrids,forms,subvarieties,sources,links.self,links.species,links.genus,main_species.id,main_species.common_name,main_species.slug,main_species.scientific_name,main_species.year,main_species.bibliography,main_species.author,main_species.status,main_species.rank,main_species.family_common_name,main_species.genus_id,main_species.observations,main_species.vegetable,main_species.image_url,main_species.genus,...,main_species.growth.light,main_species.growth.atmospheric_humidity,main_species.growth.growth_months,main_species.growth.bloom_months,main_species.growth.fruit_months,main_species.growth.minimum_precipitation.mm,main_species.growth.maximum_precipitation.mm,main_species.growth.minimum_root_depth.cm,main_species.growth.minimum_temperature.deg_f,main_species.growth.minimum_temperature.deg_c,main_species.growth.maximum_temperature.deg_f,main_species.growth.maximum_temperature.deg_c,main_species.growth.soil_nutriments,main_species.growth.soil_salinity,main_species.growth.soil_texture,main_species.growth.soil_humidity,main_species.links.self,main_species.links.plant,main_species.links.genus,genus.id,genus.name,genus.slug,genus.links.self,genus.links.plants,genus.links.species,genus.links.family,family.id,family.name,family.common_name,family.slug,family.links.self,family.links.division_order,family.links.genus,main_species.distribution.doubtful,main_species.distributions.doubtful,main_species.distribution.absent,main_species.distributions.absent,main_species.distribution.extinct,main_species.distributions.extinct,main_species
0,0,0.0,184867,stinging nettle,urtica-dioica,Urtica dioica,190500.0,https://bs.plantnet.org/image/o/85256a1c2c098e...,1753.0,Sp. Pl.: 984 (1753),L.,Nettle family,1028.0,"Europe to Siberia and W. China, NW. Africa",False,"[{'id': 190500, 'common_name': 'Stinging nettl...","[{'id': 1143593, 'common_name': 'Stinging nett...","[{'id': 190521, 'common_name': 'Stinging nettl...",[],[],[],"[{'last_update': '2020-06-21T20:55:42.484Z', '...",/api/v1/plants/urtica-dioica,/api/v1/plants/urtica-dioica/species,/api/v1/genus/urtica,190500.0,Stinging nettle,urtica-dioica,Urtica dioica,1753.0,Sp. Pl.: 984 (1753),L.,accepted,species,Nettle family,1028.0,"Europe to Siberia and W. China, NW. Africa",False,https://bs.plantnet.org/image/o/85256a1c2c098e...,Urtica,...,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,3.0,NaN,/api/v1/species/urtica-dioica,/api/v1/plants/urtica-dioica,/api/v1/genus/urtica,1028,Urtica,urtica,/api/v1/genus/urtica,/api/v1/genus/urtica/plants,/api/v1/genus/urtica/species,/api/v1/families/urticaceae,193,Urticaceae,Nettle family,urticaceae,/api/v1/families/urticaceae,/api/v1/division_orders/urticales,/api/v1/families/urticaceae/genus,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.0,121324,orchardgrass,dactylis-glomerata,Dactylis glomerata,126957.0,https://bs.plantnet.org/image/o/428f40dadfa028...,1753.0,Sp. Pl.: 71 (1753),L.,Grass family,2284.0,"Macaronesia, Medit. to Temp. Eurasia",False,"[{'id': 126957, 'common_name': 'Orchardgrass',...","[{'id': 973756, 'common_name': 'Orchardgrass',...",[],[],[],[],"[{'last_update': '2020-06-21T20:54:52.110Z', '...",/api/v1/plants/dactylis-glomerata,/api/v1/plants/dactylis-glomerata/species,/api/v1/genus/dactylis,126957.0,Orchardgrass,dactylis-glomerata,Dactylis glomerata,1753.0,Sp. Pl.: 71 (1753),L.,accepted,species,Grass family,2284.0,"Macaronesia, Medit. to Temp. Eurasia",False,https://bs.plantnet.org/image/o/428f40dadfa028...,Dactylis,...,7.0,5.0,NaN,NaN,NaN,660.0,1524.0,30.0,-22.0,-30.0,NaN,NaN,7.0,0.0,2.0,NaN,/api/v1/species/dactylis-glomerata,/api/v1/plants/dactylis-glomerata,/api/v1/genus/dactylis,2284,Dactylis,dactylis,/api/v1/genus/dactylis,/api/v1/genus/dactylis/plants,/api/v1/genus/dactylis/species,/api/v1/families/poaceae,21,Poaceae,Grass family,poaceae,/api/v1/families/poaceae,/

## Got all the column names:

In [ ]:
nw_plants_columns = nw_plants.columns.values.tolist()
df_columns = pd.DataFrame(nw_plants_columns)

In [ ]:
df_columns.to_csv('nw_columns.csv')